## Faster Calculations Via In-Place Operations

In this tutorial you will learn how to define your problems in the in-place style in order to increase your performance. First, a bit of an introduction. When solving large systems of equations, one of the largest performance hits comes from memory allocation, i.e. making arrays. Although DifferentialEquations.jl has been painstakingly optimized to prevent allocations within its solvers, allocations can also occur due to the user chosen function.

Obviously using a command like `arr = Vector{Float64}(500)` inside of your function will cause an allocation. However, there are many cases which are more subtle, for example `A.*B.*C + sin(3D)` will have make a temporary array to save the output of `A.*B`, and then use that to multiply `C`, etc. Julia's core devs are optimizing the compiler to automatically reduce allocations in events like these, though you should be careful to double-check and try devectorizing to see if it reduces allocations.

However, in-place function calls are designed to reduce an allocation that is due to an array that is returned from a function. For example, if we define a function `f(u,t) = u.*zeros(4)*t`, then if we have a pre-allocated vector `k = Vector{Float64}(4)`, we may wish to have the output of `f` write directly to `k`. But the standard way, `k = f(u,t)`, will first make an array inside of `f`, and then copy it over.

But with an in-place function, a user can get rid of that allocation. Using the in-place syntax, a user could instead define `f(du,u,t) = (du[:] = u.*zeros(4)*t)`. Notice that in this case, `du` is the output of the function, and it is given to the function, and within the function you simply write all of your output to `du`. Then to use this function without allocating arrays, someone (DifferentialEquations.jl) can call `f(k,u,t)` which will replace the values of `k` with the values of `u.*zeros(4)*t`. By doing this, there is no array that had to be allocated, saving precious time and speeding up our calculations!

## Example

Let's demonstrate this using DifferentialEquations.jl. All of the ODE and SDE solvers provided by DifferentialEquations.jl allow for the given function to be in-place. In fact, if you give DifferentialEquations.jl a function which is not in-place, it does an auto-conversion to an in-place function (which is still not as optimal as you can make yourself). Let's define a problem which holds the data for the famous [Lorenz System](https://en.wikipedia.org/wiki/Lorenz_system).

In [1]:
using DifferentialEquations
σ=10.; ρ=28.; β=8//3; u₀=ones(3)
f₁(u,t) = σ*(u[2]-u[1])
f₂(u,t) = u[1]*(ρ-u[3]) - u[2]
f₃(u,t) = u[1]*u[2] - β*u[3]
f(u,t) = [f₁(u,t);f₂(u,t);f₃(u,t)]
prob = ODEProblem(f,u₀)

DifferentialEquations.ODEProblem(f,[1.0,1.0,1.0],(anonymous function),false,3,(3,),false)

What we did was create a function `f` which returns a vector `[f1 f2 f3]`. This is the standard way to define such an `f`, and one would get the output of `f` as `output=f(u,t)`. Let's also define the problem using an in-place function:

In [2]:
function g(du,u,t)
    du[1] = σ*(u[2]-u[1])
    du[2] = u[1]*(ρ-u[3]) - u[2]
    du[3] = u[1]*u[2] - β*u[3]
end
prob2 = ODEProblem(g,u₀)

DifferentialEquations.ODEProblem(g,[1.0,1.0,1.0],(anonymous function),false,3,(3,),true)

Notice that `du` is essentially just a storage for the output. You can be more fancy with it, though in many cases this works very efficiently. Let's see the difference between the two solutions:

In [8]:
# Run once to compile
sol =solve(prob::ODEProblem,[0,1];Δt=1//2^(4),save_timeseries=false,alg=:RK4) 
sol2 =solve(prob2::ODEProblem,[0,1];Δt=1//2^(4),save_timeseries=false,alg=:RK4)
# Time the second run
@time sol =solve(prob::ODEProblem,[0,1];Δt=1//2^(6),save_timeseries=false,alg=:RK4)
@time sol2 =solve(prob2::ODEProblem,[0,1];Δt=1//2^(6),save_timeseries=false,alg=:RK4);

  0.000895 seconds (9.60 k allocations: 203.656 KB)
  0.000585 seconds (10.11 k allocations: 191.594 KB)


In [10]:
println(@allocated sol =solve(prob::ODEProblem,[0,1];Δt=1//2^(6),save_timeseries=false,alg=:RK4))
println(@allocated sol2 =solve(prob2::ODEProblem,[0,1];Δt=1//2^(6),save_timeseries=false,alg=:RK4))

208384
196032


Because the computer has to spend less time searching for free memory, on avarage the one with less allocations will be faster. For an array of size 3, it's not that big of a deal, but as the arrays grow larger, the has a bigger and bigger effect. Also, as the number of function calls per step grows, like is the case for higher order methods, this becomes more of an issue. Therefore it's recommended that you supply DifferentialEquations.jl with an in-place function if you would like the best performance.